<a href="https://colab.research.google.com/github/emisoft-designs/SEEFAR-SUSTAIN/blob/academy-data-extraction/SEEFAR_SUSTAIN_(Academy_DB_Extraction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab Notebook – Data Extraction from SEEFAR Academy Database**

**Primary Goal:**

To establish a robust foundation for data analysis and automated reporting for Seefar Academy by extracting and documenting relevant SQL database tables, and recommending suitable systems for tracking key performance indicators and differentiating user groups.

**Objectives:**
1. Extract Relevant Database Tables: Extract all non-empty tables from the pathways_moodle database into individual Pandas DataFrames.

2. Centralize Data in Google Sheets: Upload the extracted relevant tables (DataFrames) as separate worksheets to a newly created Google Spreadsheet, providing an accessible link.

3. Document Data Schema: Provide a detailed schema (column names and data types) for each extracted table. [*Map and document the different columns/fields available in the dataset*]

4. Recommend Automated Reporting Solutions: Research and propose systems capable of automating reports focused on "knowledge increase indicators" and distinguishing data trends among "employers, visa facilitators, and the two different candidate groups."



**My Plan:**

* `Extract Non-Empty Table Names`: Extract the names of all non-empty tables from the 'df_non_empty_pathways_moodle' DataFrame into a list for further processing.
* `Extract Data to DataFrames`: Iterate through the list of non-empty table names, query each table from the 'pathways_moodle' database, and load its entire content into individual Pandas DataFrames.
* `Prepare Google Sheet for Upload`: Authenticate with Google Colab and gspread, then create a new Google Spreadsheet. Clear any existing sheets to ensure a clean upload environment for the extracted data.
* `Upload Data to Google Sheets`: For each extracted DataFrame, create a new worksheet in the Google Spreadsheet and upload the DataFrame's content to it. Include appropriate error handling for large tables that might exceed Google Sheets limits.
* `Document Schema`: Extract and document the schema (column names and data types) for each table successfully uploaded to Google Sheets. Present this documentation in a clear, readable format.
* `Analyze and Recommend Reporting Systems`: Based on the project objectives related to 'knowledge increase indicators' and differentiating data trends between 'employers, visa facilitators, and two different candidate groups', research and provide recommendations for suitable automated reporting systems.
* `Finally`: Provide a summary of the extracted data, the documented schema, and the recommendations for automated reporting systems, along with links to the Google Sheet and Colab notebook.

In [ ]:
!pip install pymysql sqlalchemy gspread gspread-dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from sqlalchemy import create_engine

# RDS Database credentials. This is a MySQL database
DB_USER = "dbuserread"
DB_PASSWORD = userdata.get('db_password')
DB_HOST = "ls-17122ea7e3a528fd292a260b6217b006cb7a0f38.cyhh372xsm2h.ap-southeast-1.rds.amazonaws.com"
DB_PORT = 3306


# Define the connection URL
DATABASE_URL = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}"

# Create the engine
engine = create_engine(DATABASE_URL)

In [ ]:
import pandas as pd

# This is a function is for fetching data from the database
def fetch_data(engine, query: str = "SHOW DATABASES") -> pd.DataFrame:
  """
  Fetches data from the database using the provided query.

  Args:
      engine (sqlalchemy.engine.base.Engine): The database engine to use for querying.
      query (str, optional): The SQL query to execute. Defaults to "SHOW DATABASES".

  Returns:
      pd.DataFrame: A Pandas DataFrame containing the fetched data.
  """

  try:
      # Connect and run query
      conn = engine.connect()
      print("---> Database Connection Established")
      data = pd.read_sql(query, conn)
      print(f"Feteched data shape -> rows: {data.shape[0]} columns: {data.shape[1]}")
  except Exception as e:
      print(f"Connection failed due to error: {e}")
  finally:
      # Close the connection
      if 'conn' in locals() and conn is not None:
          conn.close()
          print("---> Database connection closed")

  # return fetched data
  return data


In [ ]:
# Let us check the tables in the database
df = fetch_data(engine, "SHOW DATABASES")
table_names = df.iloc[:, 0].tolist()
print(f"Retrieved {len(table_names)} table names from database")
print("List of Table names:")
print(table_names)

---> Database Connection Established
Feteched data shape -> rows: 3 columns: 1
---> Database connection closed
Retrieved 3 table names from database
List of Table names:
['information_schema', 'pathways_moodle', 'performance_schema']


## Explanation
From the database, we can see three databases: `information_schema, pathways_moodle`, and `performance_schema`. The `pathways_moodle` database is likely the one we are interested in.

Let's clarify the purpose of the other two databases for both technical and non-technical personnel:

* `information_schema`: This is like a data dictionary or a 'map of the database'.
  - It provides access to database metadata, including information about tables, columns, data types, access privileges, and other database objects. It's a standard feature in SQL databases for introspection and administration.
  - Think of this as the database's administrative office. It doesn't hold our actual business data, but it keeps records about all the tables, columns, users, and privileges in the entire database system. It tells us what databases exist, what tables are in them, and who can access what.

* `performance_schema`: This is designed for monitoring database performance.
  - It provides a way to examine internal execution details of the server at a low level. It collects statistics on server events (like SQL statements, I/O, locks) and resource consumption, which is invaluable for performance tuning and troubleshooting.
  - Imagine this as the database's diagnostic center. It collects data about how the database is performing – for example, how long queries are taking, what resources are being used, and if there are any bottlenecks. This helps engineers identify and fix performance issues.

Next, We will analyze the information schema to understand the tables according to our objectives and relevant data



In [ ]:
information_schema_df = fetch_data(engine, "SHOW TABLES FROM information_schema")
display(information_schema_df)

---> Database Connection Established
Feteched data shape -> rows: 81 columns: 1
---> Database connection closed


,Tables_in_information_schema
0,ADMINISTRABLE_ROLE_AUTHORIZATIONS
1,APPLICABLE_ROLES
2,CHARACTER_SETS
3,CHECK_CONSTRAINTS
4,COLLATIONS
...,...
76,USER_ATTRIBUTES
77,USER_PRIVILEGES
78,VIEWS
79,VIEW_ROUTINE_USAGE


In [ ]:
# Let us see all 81 rows containing names of the tables
display(information_schema_df.iloc[:, 0].tolist())

['ADMINISTRABLE_ROLE_AUTHORIZATIONS',
 'APPLICABLE_ROLES',
 'CHARACTER_SETS',
 'CHECK_CONSTRAINTS',
 'COLLATIONS',
 'COLLATION_CHARACTER_SET_APPLICABILITY',
 'COLUMNS',
 'COLUMNS_EXTENSIONS',
 'COLUMN_PRIVILEGES',
 'COLUMN_STATISTICS',
 'ENABLED_ROLES',
 'ENGINES',
 'EVENTS',
 'FILES',
 'INNODB_BUFFER_PAGE',
 'INNODB_BUFFER_PAGE_LRU',
 'INNODB_BUFFER_POOL_STATS',
 'INNODB_CACHED_INDEXES',
 'INNODB_CMP',
 'INNODB_CMPMEM',
 'INNODB_CMPMEM_RESET',
 'INNODB_CMP_PER_INDEX',
 'INNODB_CMP_PER_INDEX_RESET',
 'INNODB_CMP_RESET',
 'INNODB_COLUMNS',
 'INNODB_DATAFILES',
 'INNODB_FIELDS',
 'INNODB_FOREIGN',
 'INNODB_FOREIGN_COLS',
 'INNODB_FT_BEING_DELETED',
 'INNODB_FT_CONFIG',
 'INNODB_FT_DEFAULT_STOPWORD',
 'INNODB_FT_DELETED',
 'INNODB_FT_INDEX_CACHE',
 'INNODB_FT_INDEX_TABLE',
 'INNODB_INDEXES',
 'INNODB_METRICS',
 'INNODB_SESSION_TEMP_TABLESPACES',
 'INNODB_TABLES',
 'INNODB_TABLESPACES',
 'INNODB_TABLESPACES_BRIEF',
 'INNODB_TABLESTATS',
 'INNODB_TEMP_TABLE_INFO',
 'INNODB_TRX',
 'INNODB_VI

In [ ]:
# Now let us query the information_schema table for specific table (`pathways_moodle`)
# and enquire some certain details like get all the table names, their types and length
# of data in them.
query_info_schema = """
SELECT
  table_name,
  table_type,
  engine,
  table_rows,
  avg_row_length,
  data_length,
  index_length,
  table_comment
FROM
  information_schema.tables
WHERE
  table_schema = 'pathways_moodle'
ORDER BY
  table_name;
"""
df_pathways_moodle_info = fetch_data(engine, query_info_schema)
display(df_pathways_moodle_info)

---> Database Connection Established
Feteched data shape -> rows: 511 columns: 8
---> Database connection closed


,TABLE_NAME,TABLE_TYPE,ENGINE,TABLE_ROWS,AVG_ROW_LENGTH,DATA_LENGTH,INDEX_LENGTH,TABLE_COMMENT
0,mdl_adminpresets,BASE TABLE,InnoDB,2.0,4096.0,8192.0,0.0,Table to store presets data
1,mdl_adminpresets_app,BASE TABLE,InnoDB,0.0,0.0,8192.0,8192.0,Applied presets
2,mdl_adminpresets_app_it,BASE TABLE,InnoDB,0.0,0.0,8192.0,16384.0,Admin presets applied items. To maintain the r...
3,mdl_adminpresets_app_it_a,BASE TABLE,InnoDB,0.0,0.0,8192.0,16384.0,Attributes of the applied items
4,mdl_adminpresets_app_plug,BASE TABLE,InnoDB,0.0,0.0,8192.0,8192.0,Admin presets plugins applied
...,...,...,...,...,...,...,...,...
506,mdl_workshopform_rubric,BASE TABLE,InnoDB,0.0,0.0,8192.0,8192.0,The assessment dimensions definitions of Rubri...
507,mdl_workshopform_rubric_config,BASE TABLE,InnoDB,0.0,0.0,8192.0,8192.0,Configuration table for the Rubric grading str...
508,mdl_workshopform_rubric_levels,BASE TABLE,InnoDB,0.0,0.0,8192.0,8192.0,The definition of rubric rating scales
509,mdl_xapi_states,BASE TABLE,InnoDB,0.0,0.0,8192.0,24576.0,The stored xAPI states


## Explanation:

This query provides a comprehensive overview of each table in the `pathways_moodle` database, including its type (BASE TABLE or VIEW), storage engine, estimated number of rows, average row length, and sizes of data and indexes. The `table_comment` column, if populated, can also offer valuable insights into the table's purpose.

As you can see we have about 511 tables in the `pathways_moodle` table. If you noticed some of these tables are empty, it is likely they are not queried or updated from the website, which means users are not querying these tables, so let us put our focus on the main goal which we would likely get from the table users are using. I would like us to dig further to select the tables that are non-empty.

In [ ]:
# Filter for non-empty tables
df_non_empty_pathways_moodle = df_pathways_moodle_info[df_pathways_moodle_info['TABLE_ROWS'] > 0]
display(f"Table Shape -> Rows: {df_non_empty_pathways_moodle.shape[0]}, Columns: {df_non_empty_pathways_moodle.shape[1]}")
display(df_non_empty_pathways_moodle)

'Table Shape -> Rows: 206, Columns: 8'

,TABLE_NAME,TABLE_TYPE,ENGINE,TABLE_ROWS,AVG_ROW_LENGTH,DATA_LENGTH,INDEX_LENGTH,TABLE_COMMENT
0,mdl_adminpresets,BASE TABLE,InnoDB,2.0,4096.0,8192.0,0.0,Table to store presets data
5,mdl_adminpresets_it,BASE TABLE,InnoDB,24.0,341.0,8192.0,8192.0,Table to store settings
7,mdl_adminpresets_plug,BASE TABLE,InnoDB,115.0,71.0,8192.0,8192.0,"Admin presets plugins status, to store informa..."
8,mdl_analytics_indicator_calc,BASE TABLE,InnoDB,63.0,130.0,8192.0,16384.0,Stored indicator calculations
9,mdl_analytics_models,BASE TABLE,InnoDB,5.0,1638.0,8192.0,16384.0,Analytic models.
...,...,...,...,...,...,...,...,...
481,mdl_user_info_data,BASE TABLE,InnoDB,6972.0,610.0,4259840.0,155648.0,Data for the customisable user fields
482,mdl_user_info_field,BASE TABLE,InnoDB,4.0,2048.0,8192.0,0.0,Customisable user profile fields
483,mdl_user_lastaccess,BASE TABLE,InnoDB,1544.0,37.0,57344.0,114688.0,"To keep track of course page access times, use..."
486,mdl_user_preferences,BASE TABLE,InnoDB,5896.0,37.0,221184.0,352256.0,Allows modules to store arbitrary user prefere...


## Explanation:
This table now shows only the non-empty tables from the `pathways_moodle` database. The `TABLE_ROWS` column indicates the number of rows in each table, and by filtering for `TABLE_ROWS > 0`, we ensure that we are looking at tables that actually contain data. This helps in identifying potentially important tables for further analysis. So far there are 206 non - empty tables in the `pathways_moodle` DataTable. Now let me extract the names of these tables, so we can dig deeper.

In [ ]:
# Give us the list of tables name all 206 tables
active_table_names = df_non_empty_pathways_moodle['TABLE_NAME'].to_list()
display(active_table_names)

['mdl_adminpresets',
 'mdl_adminpresets_it',
 'mdl_adminpresets_plug',
 'mdl_analytics_indicator_calc',
 'mdl_analytics_models',
 'mdl_analytics_predict_samples',
 'mdl_analytics_prediction_actions',
 'mdl_analytics_predictions',
 'mdl_analytics_used_analysables',
 'mdl_assign',
 'mdl_assign_grades',
 'mdl_assign_plugin_config',
 'mdl_assign_submission',
 'mdl_assign_user_mapping',
 'mdl_assignfeedback_editpdf_rot',
 'mdl_assignsubmission_file',
 'mdl_assignsubmission_onlinetext',
 'mdl_auth_oauth2_linked_login',
 'mdl_backup_controllers',
 'mdl_block',
 'mdl_block_configurable_reports',
 'mdl_block_instances',
 'mdl_block_positions',
 'mdl_block_recentlyaccesseditems',
 'mdl_book',
 'mdl_book_chapters',
 'mdl_capabilities',
 'mdl_choice',
 'mdl_choice_answers',
 'mdl_choice_options',
 'mdl_cohort_members',
 'mdl_config',
 'mdl_config_log',
 'mdl_config_plugins',
 'mdl_contentbank_content',
 'mdl_context',
 'mdl_course',
 'mdl_course_categories',
 'mdl_course_completion_aggr_methd',
 '

Now we have successfully extracted and displayed a list of all the non-empty table names from the pathways_moodle database. This list contains 206 tables, which are the ones that actually hold data relevant to our analysis.

## Explanation:
* **What we did**: We previously identified that the `pathways_moodle` database contains a total of 511 tables. Some of these tables were empty, meaning they didn't contain any useful information. In the last step, we filtered out all the empty tables and extracted a list of only those tables that actually contain data.

* **What this means**:
  - **Technical**: We have retrieved the `TABLE_NAME` column from the `df_non_empty_pathways_moodle` DataFrame, which was previously filtered to include only tables where `TABLE_ROWS > 0`. This result is a Python list of 206 strings, where each string is the name of a table within the `pathways_moodle` schema that contains at least one row of data.

  - **Non-Technical**: Think of the database as a huge library with many books (tables). Some books are completely empty. What we've done is create a precise list of only the books that have actual content in them. This list contains 206 "content-filled" books that are important for Seefar Academy. These are the main sources of information about users, courses, and activities.

This list is crucial as it guides the subsequent data extraction process, ensuring we only query and process tables that hold meaningful information, optimizing resources and focusing on relevant data assets.

## **What Next:**
Now that we have this definitive list of active tables, our next step will be to go into each of these 206 tables, pull out all their data, and organize it into separate DataFrames. This is a critical step towards our goal of documenting the data and making it available for mapping and reporting.

So we:

1. **Extract Data to DataFrames**: Iterate through the list of non-empty table names, query each table from the 'pathways_moodle' database, and load its entire content into individual Pandas DataFrames.

2. **Prepare Google Sheet for Upload**: Authenticate with Google Colab and gspread, then create a new Google Spreadsheet. Clear any existing sheets to ensure a clean upload environment for the extracted data.

3. **Upload Data to Google Sheets**: For each extracted DataFrame, create a new worksheet in the Google Spreadsheet and upload the DataFrame's content to it. Include appropriate error handling for large tables that might exceed Google Sheets limits.

4. **Document Schema**: Extract and document the schema (column names and data types) for each table successfully uploaded to Google Sheets. Present this documentation in a clear, readable format.

5. **Analyze and Recommend Reporting Systems**: Based on the project objectives related to 'knowledge increase indicators' and differentiating data trends between 'employers, visa facilitators, and two different candidate groups', research and provide recommendations for suitable automated reporting systems.

6. **Finally**: Provide a summary of the extracted data, the documented schema, and the recommendations for automated reporting systems, along with links to the Google Sheet and Colab notebook



In [ ]:
from typing import Dict, Any

# A dictionary to store all the extracted tables as dataframes.
# This would aid future selection and storage
active_tables: Dict[str, Any] = {}

def extract_active_tables_to_df(engine, table_names):
  """
  Extracts and loads data from specified tables into Pandas DataFrames.

  Args:
      engine (sqlalchemy.engine.base.Engine): The database engine to use for querying.
      table_names (List[str]): A list of table names to extract and load into DataFrames.

  Returns:
      Dict[str, pd.DataFrame]: A dictionary where keys are table names and values are
                              corresponding Pandas DataFrames

  """
  tables = {}  # temporary storage for active tables
  for table_name in table_names:
      print(f"\n\n---> Displaying data for table: {table_name} ---\n")
      query = f"SELECT * FROM pathways_moodle.{table_name}"
      try:
          tables[table_name] = fetch_data(engine, query)
          display(tables[table_name].head(5))
          print(f"Successfully extracted data for table: {table_name}")
      except Exception as e:
          print(f"Error reading table {table_name}: {e}")
          continue
  return tables

active_tables = extract_active_tables_to_df(engine, active_table_names)



---> Displaying data for table: mdl_adminpresets ---

---> Database Connection Established
Feteched data shape -> rows: 2 columns: 11
---> Database connection closed


,id,userid,name,comments,site,author,moodleversion,moodlerelease,iscore,timecreated,timeimported
0,1,0,Starter,"Moodle with all of the most popular features, ...",http://localhost:8080,,,,1,1693378309,0
1,2,0,Full,All the Starter features plus External (LTI) t...,http://localhost:8080,,,,2,1693378310,0


Successfully extracted data for table: mdl_adminpresets


---> Displaying data for table: mdl_adminpresets_it ---

---> Database Connection Established
Feteched data shape -> rows: 24 columns: 5
---> Database connection closed


,id,adminpresetid,plugin,name,value
0,1,1,none,usecomments,0
1,2,1,none,usetags,0
2,3,1,none,enablenotes,0
3,4,1,none,enableblogs,0
4,5,1,none,enablebadges,0


Successfully extracted data for table: mdl_adminpresets_it


---> Displaying data for table: mdl_adminpresets_plug ---

---> Database Connection Established
Feteched data shape -> rows: 115 columns: 5
---> Database connection closed


...

Successfully extracted data for table: mdl_user_lastaccess


---> Displaying data for table: mdl_user_preferences ---

---> Database Connection Established
Feteched data shape -> rows: 5705 columns: 4
---> Database connection closed


,id,userid,name,value
0,1,2,core_message_migrate_data,1
1,2,2,core_user_welcome,1693378494
2,3,2,login_failed_count_since_success,487
3,4,2,tool_usertours_tour_completion_time_4,1693379061
4,5,2,drawer-open-index,0


Successfully extracted data for table: mdl_user_preferences


---> Displaying data for table: mdl_user_private_key ---

---> Database Connection Established
Feteched data shape -> rows: 1041 columns: 8
---> Database connection closed


,id,script,value,userid,instance,iprestriction,validuntil,timecreated
0,1,core_files,a90b0cd896ec5de051901b2ee571f986,2,None,None,None,1693453982
1,2,core_files,6c012dbbc7b2915cee79a5ab890fc678,11,None,None,None,1696599482
2,5,core_files,5ac86c5e32616385e81825170f236714,17,None,None,None,1698780753
3,6,core_files,281a830c707db7e732611d50943dfe44,33,None,None,None,1699302990
4,7,core_files,5215b7c8ab2bd4a0fb829a4937d120c1,36,None,None,None,1699426959


Successfully extracted data for table: mdl_user_private_key


## Explanation:

Now we have successfully loaded and displayed tables, but we have some issue with some of the tables, they do not contain the data we need for the core task and that are user eccentric. Most of these data are system config, log, and meta data. We do not want to include this, so I would take it step by step to remove all the meta data and curate a list of these data we do not want.

In [ ]:
# List of tables identified for removal (metadata, event logs, configuration, or not directly user-centric for knowledge increase)
tables_to_remove = [
    'mdl_adminpresets',
    'mdl_adminpresets_it',
    'mdl_adminpresets_plug',
    'mdl_analytics_indicator_calc',
    'mdl_analytics_models',
    'mdl_analytics_used_analysables',
    'mdl_auth_oauth2_linked_login',
    'mdl_backup_controllers',
    'mdl_block',
    'mdl_block_configurable_reports',
    'mdl_block_instances',
    'mdl_block_positions',
    'mdl_block_recentlyaccesseditems',
    'mdl_capabilities',
    'mdl_config',
    'mdl_config_log',
    'mdl_config_plugins',
    'mdl_context',
    'mdl_event',
    'mdl_external_functions',
    'mdl_external_services',
    'mdl_external_services_functions',
    'mdl_external_tokens',
    'mdl_favourite',
    'mdl_files',
    'mdl_files_reference',
    'mdl_filter_active',
    'mdl_glossary_formats',
    'mdl_grading_areas',
    'mdl_h5p_contents_libraries',
    'mdl_h5p_libraries',
    'mdl_h5p_libraries_cachedassets',
    'mdl_h5p_library_dependencies',
    'mdl_license',
    'mdl_local_profilecohort',
    'mdl_log_display',
    'mdl_logstore_standard_log',
    'mdl_message_contact_requests',
    'mdl_message_conversation_members',
    'mdl_message_conversations',
    'mdl_message_popup_notifications',
    'mdl_message_processors',
    'mdl_message_providers',
    'mdl_message_user_actions',
    'mdl_messageinbound_handlers',
    'mdl_messages',
    'mdl_mnet_application',
    'mdl_mnet_host',
    'mdl_mnet_remote_rpc',
    'mdl_mnet_remote_service2rpc',
    'mdl_mnet_rpc',
    'mdl_mnet_service',
    'mdl_mnet_service2rpc',
    'mdl_modules',
    'mdl_my_pages',
    'mdl_notifications',
    'mdl_oauth2_endpoint',
    'mdl_oauth2_issuer',
    'mdl_oauth2_user_field_mapping',
    'mdl_quiz_reports',
    'mdl_report_customsql_categories',
    'mdl_report_customsql_queries',
    'mdl_reportbuilder_column',
    'mdl_reportbuilder_filter',
    'mdl_reportbuilder_report',
    'mdl_repository',
    'mdl_repository_instances',
    'mdl_role',
    'mdl_role_allow_assign',
    'mdl_role_allow_override',
    'mdl_role_allow_switch',
    'mdl_role_allow_view',
    'mdl_role_assignments',
    'mdl_role_capabilities',
    'mdl_role_context_levels',
    'mdl_scale',
    'mdl_search_simpledb_index',
    'mdl_sessions',
    'mdl_stats_daily',
    'mdl_stats_monthly',
    'mdl_stats_user_daily',
    'mdl_stats_user_monthly',
    'mdl_stats_user_weekly',
    'mdl_stats_weekly',
    'mdl_tag',
    'mdl_tag_area',
    'mdl_tag_instance',
    'mdl_task_log',
    'mdl_task_scheduled',
    'mdl_tiny_autosave',
    'mdl_tool_brickfield_checks',
    'mdl_tool_customlang',
    'mdl_tool_customlang_components',
    'mdl_tool_dataprivacy_request',
    'mdl_tool_mfa',
    'mdl_tool_mfa_auth',
    'mdl_tool_usertours_steps',
    'mdl_tool_usertours_tours',
    'mdl_upgrade_log',
    'mdl_user_info_category',
    'mdl_user_lastaccess',
    'mdl_user_preferences',
    'mdl_user_private_key'
]

must_keep_tables = [
    "mdl_user",
    "mdl_user_info_field",
    "mdl_user_info_data",
    "mdl_course",
    "mdl_course_categories",
    "mdl_enrol",
    "mdl_user_enrolments",
    "mdl_cohort_members",
    "mdl_logstore_standard_log",
    "mdl_course_completions",
    "mdl_quiz",
    "mdl_quiz_attempts",
    "mdl_quiz_grades",
    "mdl_feedback",
    "mdl_feedback_completed",
    "mdl_feedback_item",
    "mdl_feedback_value",
    "mdl_role",
    "mdl_role_assignments",
    "mdl_context",
]

# updated tables to remove:
tables_to_remove = [table for table in tables_to_remove if table not in must_keep_tables]

In [ ]:
from typing import Dict, List
import pandas as pd

def filter_out_non_user_tables(dataframes_dict: Dict[str, pd.DataFrame], tables_to_remove: List[str]) -> Dict[str, pd.DataFrame]:
    """
    Filters a dictionary of DataFrames, removing non user specified tables.

    Args:
        dataframes_dict (Dict[str, pd.DataFrame]): A dictionary where keys are table names
                                                 and values are pandas DataFrames.
        tables_to_remove (List[str]): A list of table names to be removed.

    Returns:
        Dict[str, pd.DataFrame]: A new dictionary containing only the DataFrames
                                 whose table names are not in tables_to_remove.
    """
    filtered_dict = {name: df for name, df in dataframes_dict.items() if name not in tables_to_remove}
    return filtered_dict

# Apply the filtering function
active_tables_filtered = filter_out_non_user_tables(active_tables, tables_to_remove)

print(f"Original number of active tables: {len(active_tables)}")
print(f"Number of tables after filtering: {len(active_tables_filtered)}")
print("Remaining tables:")
for table_name in active_tables_filtered.keys():
    print(f"- {table_name}")

Original number of active tables: 206
Number of tables after filtering: 103
Remaining tables:
- mdl_analytics_predict_samples
- mdl_analytics_prediction_actions
- mdl_analytics_predictions
- mdl_assign
- mdl_assign_grades
- mdl_assign_plugin_config
- mdl_assign_submission
- mdl_assign_user_mapping
- mdl_assignfeedback_editpdf_rot
- mdl_assignsubmission_file
- mdl_assignsubmission_onlinetext
- mdl_book
- mdl_book_chapters
- mdl_choice
- mdl_choice_answers
- mdl_choice_options
- mdl_cohort_members
- mdl_contentbank_content
- mdl_course
- mdl_course_categories
- mdl_course_completion_aggr_methd
- mdl_course_completion_crit_compl
- mdl_course_completion_criteria
- mdl_course_completions
- mdl_course_format_options
- mdl_course_modules
- mdl_course_modules_completion
- mdl_course_modules_viewed
- mdl_course_sections
- mdl_customcert
- mdl_customcert_elements
- mdl_customcert_issues
- mdl_customcert_pages
- mdl_customcert_templates
- mdl_enrol
- mdl_feedback
- mdl_feedback_completed
- mdl_fe

## Explanation:
We have now filtered the `active_tables` dictionary, removing tables that were identified as metadata, system configurations, generic logs, or other tables not directly contributing to measuring user knowledge increase.

The `active_tables_filtered` dictionary now contains a more focused set of tables that are likely to hold relevant user data for our analysis goals.

Now that we have selected user related tables we noticed these are not user-eccentric tables, we need to get the tables that are directly related to the users so we can map the users table and merge their data into one dataframe for storage in google sheet. This is important so we can have a focused data and less columns. We would only select tables with `userid` column, these are tables directly relating to the users and have the users information on them.


In [ ]:
def select_user_related_tables(dataframes_dict: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
  """
  Selects tables related to users from a dictionary of DataFrames.

  Args:
      dataframes_dict (Dict[str, pd.DataFrame]): A dictionary where keys are table names
                                                 and values are pandas DataFrames.

  Returns:
      Dict[str, pd.DataFrame]: A new dictionary containing only.
  """
  users_related_tables = {name: df for name, df in dataframes_dict.items() if (df is not None
                                                                      and not df.empty
                                                                      and "userid" in df.columns)
                                                                       or name == "mdl_user"}
  return users_related_tables


users_related_tables = select_user_related_tables(active_tables_filtered)

print(f"Number of tables related to users: {len(users_related_tables)} out of {len(active_tables_filtered)}")
print("Tables related to users:")
for table_name in users_related_tables.keys():
    print(f"- {table_name}")

Number of tables related to users: 22 out of 103
Tables related to users:
- mdl_analytics_prediction_actions
- mdl_assign_grades
- mdl_assign_submission
- mdl_assign_user_mapping
- mdl_choice_answers
- mdl_cohort_members
- mdl_course_completion_crit_compl
- mdl_course_completions
- mdl_course_modules_completion
- mdl_course_modules_viewed
- mdl_customcert_issues
- mdl_feedback_completed
- mdl_flashcard_card
- mdl_grade_grades
- mdl_grade_grades_history
- mdl_question_attempt_steps
- mdl_questionnaire_response
- mdl_quiz_attempts
- mdl_quiz_grades
- mdl_user
- mdl_user_enrolments
- mdl_user_info_data


In [ ]:
user_related_df = users_related_tables.copy()
display(len(user_related_df))

# drop 'mdl_users` from dict
del user_related_df['mdl_user']
display(len(user_related_df))

22

21

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("Google Colab user authenticated.")

Google Colab user authenticated.


In [ ]:
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default

# Authorize gspread to interact with Google Sheets using the authenticated user's credentials
creds, _ = default()
gc = gspread.Client(creds)
print("gspread client authorized using OAuth.")

gspread client authorized using OAuth.


In [ ]:
spreadsheet_name = 'Pathways Moodle Selected Database Tables'
spreadsheet = gc.create(spreadsheet_name)
print(f"Google Spreadsheet '{spreadsheet_name}' created successfully.")
print(f"Spreadsheet URL: {spreadsheet.url}")

Google Spreadsheet 'Pathways Moodle Selected Database Tables' created successfully.
Spreadsheet URL: https://docs.google.com/spreadsheets/d/1LIUTgVc31uC9xgMpfPM7heVSxk6IhIEN2o0z40ceGsQ


In [ ]:
import time
import gspread
from gspread_dataframe import set_with_dataframe

MAX_RETRIES = 2
SLEEP_SECONDS = 1
WORKSHEET_NAME_LIMIT = 99


def make_worksheet_name(table_name: str) -> str:
    """
    Create a worksheet-safe name from a table name.

    Usage:
        worksheet_name = make_worksheet_name("mdl_analytics_logs")
    """
    return table_name.replace("mdl_", "")[:WORKSHEET_NAME_LIMIT]



In [ ]:
def compute_sheet_dimensions(df):
    """
    Compute safe worksheet dimensions for a DataFrame.

    Usage:
        rows, cols = compute_sheet_dimensions(df)
    """
    rows = max(df.shape[0] + 1, 2)  # +1 for header
    cols = max(df.shape[1], 1)
    return rows, cols

In [ ]:
def recreate_worksheet(spreadsheet, title: str, rows: int, cols: int):
    """
    Create a worksheet, deleting any existing one with the same name.

    Usage:
        ws = recreate_worksheet(spreadsheet, "analytics_logs", 10, 5)
    """
    try:
        return spreadsheet.add_worksheet(title=title, rows=rows, cols=cols)
    except gspread.exceptions.APIError as e:
        if "already exists" in str(e):
            ws = spreadsheet.worksheet(title)
            spreadsheet.del_worksheet(ws)
            return spreadsheet.add_worksheet(title=title, rows=rows, cols=cols)
        raise

In [ ]:
def write_dataframe_to_sheet(spreadsheet, table_name: str, df):
    """
    Write a non-empty DataFrame to a Google Sheet worksheet.

    Usage:
        write_dataframe_to_sheet(spreadsheet, "mdl_users", users_df)
    """
    worksheet_name = make_worksheet_name(table_name)
    rows, cols = compute_sheet_dimensions(df)

    worksheet = recreate_worksheet(
        spreadsheet,
        title=worksheet_name,
        rows=rows,
        cols=cols,
    )

    set_with_dataframe(worksheet, df)
    time.sleep(SLEEP_SECONDS)


In [ ]:
def with_retries(fn, retries: int = MAX_RETRIES):
    """
    Execute a function with retry support.

    Usage:
        with_retries(lambda: write_dataframe_to_sheet(...))
    """
    for attempt in range(retries):
        try:
            return fn()
        except Exception as e:
            if attempt == retries - 1:
                raise
            print(f"Retrying after error: {e}")


In [ ]:
def process_tables(spreadsheet, tables: dict) -> dict:
    """
    Process multiple DataFrames and write non-empty ones to GSheets.

    Usage:
        saved_dfs = process_tables(spreadsheet, user_related_tables)
    """
    saved_dfs = {}

    for name, df in tables.items():
        print(f"Processing table: {name}")

        if df.empty:
            print(f"Table '{name}' is empty and will be skipped.")
            continue

        # infer data types
        df = df.convert_dtypes()

        try:
            with_retries(
                lambda: write_dataframe_to_sheet(spreadsheet, name, df)
            )
            saved_dfs[name] = df
            print(f"Successfully wrote '{name}'")
        except Exception as e:
            print(f"Failed processing table '{name}': {e}")

    return saved_dfs


In [ ]:
saved_dfs = process_tables(
    spreadsheet=spreadsheet,
    tables=user_related_df ,
)

print(
    f"All non-empty tables processed and saved to '{spreadsheet_name}'."
)
print(f"Spreadsheet URL: {spreadsheet.url}")


Processing table: mdl_analytics_prediction_actions
Successfully wrote 'mdl_analytics_prediction_actions'
Processing table: mdl_assign_grades
Successfully wrote 'mdl_assign_grades'
Processing table: mdl_assign_submission
Successfully wrote 'mdl_assign_submission'
Processing table: mdl_assign_user_mapping
Successfully wrote 'mdl_assign_user_mapping'
Processing table: mdl_choice_answers
Successfully wrote 'mdl_choice_answers'
Processing table: mdl_cohort_members
Successfully wrote 'mdl_cohort_members'
Processing table: mdl_course_completion_crit_compl
Successfully wrote 'mdl_course_completion_crit_compl'
Processing table: mdl_course_completions
Successfully wrote 'mdl_course_completions'
Processing table: mdl_course_modules_completion
Successfully wrote 'mdl_course_modules_completion'
Processing table: mdl_course_modules_viewed
Successfully wrote 'mdl_course_modules_viewed'
Processing table: mdl_customcert_issues
Successfully wrote 'mdl_customcert_issues'
Processing table: mdl_feedback_co